# CrewAI Coder Agent - Control Center

This notebook provides an interactive interface to control and monitor the CrewAI coder agent.

## Features
- Configure and run the agent
- Monitor execution in real-time
- View logs and metrics
- Customize prompts and settings
- Inspect generated files

## Setup and Imports

In [26]:
# Imports
import sys
import time
from datetime import datetime
from pathlib import Path

from helpers.logger import SessionLogger
from helpers.config_loader import load_config
from core.agent_factory import create_coder_crew
from core.context_loader import load_prompt_template

print("✓ All imports successful")

✓ All imports successful


## Load Configuration

In [27]:
# Load config
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

Configuration:
  LLM Model: qwen/qwen3-coder-30b
  Base URL: http://localhost:1234/v1
  Temperature: 0.7
  Max Tokens: 4000
  Agent Role: Senior Full-Stack Developer
  Max Iterations: 25


## Initialize Session Logger

In [28]:
# Create session logger
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = SessionLogger(session_id=session_id)

logger.print_success(f"Session started: {session_id}")
logger.print_info(f"Logs will be saved to: logs/{session_id}.log")

✓ Session started: notebook_20251109_153634

ℹ Logs will be saved to: logs/notebook_20251109_153634.log

## View Default Prompt

In [29]:
# Load and display the default prompt
default_prompt = load_prompt_template('build-website')
print("Default Task Prompt:")
print("=" * 80)
print(default_prompt[:1000])  # Show first 1000 characters
print("...")
print(f"\nTotal length: {len(default_prompt)} characters")

Default Task Prompt:
# Build Personal Website - Agent Task

## Objective
Create a modern, responsive personal portfolio website using React and Tailwind CSS.

## Requirements

### Technical Stack
- React 18+
- Tailwind CSS for styling
- Vite as build tool
- Modern ES6+ JavaScript
- Responsive design (mobile-first)

### Features to Implement

#### 1. Navigation
- Sticky navigation bar
- Smooth scroll to sections
- Mobile hamburger menu
- Active section highlighting

#### 2. Hero Section
- Full-screen hero with gradient background
- Name and title/tagline
- Call-to-action button
- Subtle animations on load

#### 3. About Section
- Personal introduction (2-3 paragraphs)
- Professional background
- Current work/interests
- Optional: profile image with hover effect

#### 4. Skills Section
- Grid layout of technical skills
- Categorized skills (Frontend, Backend, Tools, etc.)
- Visual indicators (icons or badges)
- Hover effects

#### 5. Portfolio/Projects Section
- Grid of project cards (mi

## Option 1: Run with Default Prompt

In [30]:
# Create and run crew with default settings
logger.print_section("Creating CrewAI Crew")

crew = create_coder_crew(verbose=True)

logger.print_success("Crew created successfully")
logger.print_info(f"Agents: {len(crew.agents)}")
logger.print_info(f"Tasks: {len(crew.tasks)}")

────────────────────────────────────────────── Creating CrewAI Crew ───────────────────────────────────────────────

✓ Crew created successfully

ℹ Agents: 1

ℹ Tasks: 1

In [31]:
# Execute the crew
logger.print_section("Executing Task")
logger.print_info("Building personal website...")
logger.print_info("This may take several minutes.\n")

start_time = time.time()

try:
    result = crew.kickoff()
    
    execution_time = time.time() - start_time
    
    logger.print_section("Execution Complete")
    logger.print_success(f"Completed in {execution_time:.2f} seconds")
    
    print("\nResult:")
    print("=" * 80)
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

───────────────────────────────────────────────── Executing Task ──────────────────────────────────────────────────

ℹ Building personal website...

ℹ This may take several minutes.

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4bb745b5-da86-4d86-a406-945f096ad335                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Full-Stack Developer                                                                             │
│                                                                                                                 │
│  Task: # Build Personal Website - Agent Task                                                                    │
│                                                                                                                 │
│  ## Objective                                                                                                   │
│  Create a modern, responsive personal portfolio website using React and Tailwind CSS.                           │
│                                                                                                                 │
│  ## Requirements                                                                                                │
│                                                                                                                 │
│  ### Technical Stack                                                                                            │
│  - React 18+                                                                                                    │
│  - Tailwind CSS for styling                                                                                     │
│  - Vite as build tool                                                                                           │
│  - Modern ES6+ JavaScript                                                                                       │
│  - Responsive design (mobile-first)                                                                             │
│                                                                                                                 │
│  ### Features to Implement                                                                                      │
│                                                                                                                 │
│  #### 1. Navigation                                                                                             │
│  - Sticky navigation bar                                                                                        │
│  - Smooth scroll to sections                                                                                    │
│  - Mobile hamburger menu                                                                                        │
│  - Active section highlighting                                                                                  │
│                                                                                                                 │
│  #### 2. Hero Section                                                                                           │
│  - Full-screen hero with gradient background                                                                    │
│  - Name and title/tagline                                                                                       │
│  - Call-to-action button                                                                                        │
│  - Subtle animations on load                                                                                    │
│                                                                                                                 │
│  #### 3. About Section                                                                                          │
│  - Personal introduction (2-3 paragraphs)                                                                       │
│  - Professional background                             

...

✗ Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You 
passed model=qwen/qwen3-coder-30b
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` 
Learn more: https://docs.litellm.ai/docs/providers

Would you like to view your execution traces? [y/N] (20s timeout): 

Traceback (most recent call last):
  File "/var/folders/sz/qdyj3ffd5rxg1vjk8d82pkkm0000gp/T/ipykernel_24458/1118292581.py", line 9, in <module>
    result = crew.kickoff()
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/crew.py", line 705, in kickoff
    result = self._run_sequential_process()
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/crew.py", line 821, in _run_sequential_process
    return self._execute_tasks(self.tasks)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/crew.py", line 929, in _execute_tasks
    task_output = task.execute_sync(
        agent=agent_to_use,
        context=context,
        tools=tools_for_task,
    )
  File "/Users/Khaled.Alabsi/projects/llm-agent/.venv/lib/python3.13/site-packages/crewai/task.py", line 458, in execute_sync
    return self._execute_core(agent, context, tools)
       

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 6c418bc7-3760-4cd1-8226-fcbef119689f                                                                     │
│  Agent: Senior Full-Stack Developer                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=qwen/qwen3-coder-30b                                                                          │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Option 2: Run with Custom Prompt

In [ ]:
# Define custom prompt
custom_prompt = """
Create a minimalist personal portfolio website with the following requirements:

Technical Stack:
- React 18 with functional components
- Tailwind CSS for styling
- Vite as build tool

Sections:
1. Hero section with name and title
2. About section
3. Skills grid
4. Project showcase (3 projects minimum)
5. Contact form

Requirements:
- Fully responsive design
- Dark mode toggle
- Smooth scrolling navigation
- Modern, clean aesthetic
- All code must be well-documented
- Include package.json and README

Output everything to ./output directory.
"""

print("Custom Prompt:")
print(custom_prompt)

In [ ]:
# Create crew with custom prompt
custom_crew = create_coder_crew(custom_prompt=custom_prompt, verbose=True)

logger.print_success("Custom crew created")

In [ ]:
# Execute custom crew
start_time = time.time()

try:
    result = custom_crew.kickoff()
    
    execution_time = time.time() - start_time
    logger.print_success(f"Custom task completed in {execution_time:.2f} seconds")
    
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

## Inspect Generated Files

In [ ]:
# List all files in output directory
import os

output_dir = Path("./output")

if output_dir.exists():
    print("Generated Files:")
    print("=" * 80)
    
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(str(output_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = Path(root) / file
            size = file_path.stat().st_size
            print(f"{subindent}{file} ({size:,} bytes)")
else:
    print("Output directory not found or empty")

## View Specific Files

In [ ]:
# Read and display a specific file
file_to_view = "output/package.json"  # Change this to view different files

try:
    with open(file_to_view, 'r') as f:
        content = f.read()
    
    print(f"File: {file_to_view}")
    print("=" * 80)
    print(content)
    
except FileNotFoundError:
    print(f"File not found: {file_to_view}")
except Exception as e:
    print(f"Error reading file: {e}")

## View Session Logs

In [ ]:
# View session metrics
import json

json_log = Path(f"logs/{session_id}.json")

if json_log.exists():
    with open(json_log, 'r') as f:
        session_data = json.load(f)
    
    print("Session Metrics:")
    print("=" * 80)
    print(json.dumps(session_data.get('metrics', {}), indent=2))
    
    print("\nRecent Events:")
    print("=" * 80)
    events = session_data.get('events', [])
    for event in events[-5:]:  # Show last 5 events
        print(f"\n[{event['timestamp']}] {event['type']}")
        if 'data' in event:
            print(json.dumps(event['data'], indent=2)[:200])
else:
    print("Session log not found yet")

## End Session

In [ ]:
# End session and save final logs
logger.end_session()

print("\n✓ Session ended. Logs saved.")

## Quick Reference

### Common Tasks

1. **Run with default settings**: Execute the first "Option 1" cells
2. **Run with custom prompt**: Modify and execute "Option 2" cells
3. **Check output**: Run "Inspect Generated Files" cell
4. **View logs**: Run "View Session Logs" cell

### Configuration Changes

Edit `config.yaml` to change:
- LLM settings (model, temperature, etc.)
- Agent behavior (role, goal, max iterations)
- Output directory
- Logging settings

### Context Files

Modify these to change agent behavior:
- `context/no-goes.md` - Safety rules
- `context/guidelines/coding-standards.md` - Code standards
- `context/skills/` - Knowledge base
- `prompts/build-website.md` - Default task prompt

### Output Directory

All generated files go to `./output/`
This directory is gitignored but tracked for changes.

### Logs Directory

Session logs are saved to `./logs/`
- `session_*.log` - Human-readable log
- `session_*.json` - Structured data with metrics


In [ ]:
# Cell 1: Imports
import sys
import time
from datetime import datetime
from pathlib import Path

from helpers.logger import SessionLogger
from helpers.config_loader import load_config
from core.agent_factory import create_coder_crew
from core.context_loader import load_prompt_template

print("✓ All imports successful")

In [ ]:
# Cell 2: Load Configuration
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

In [ ]:
# Cell 2: Load Configuration
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

In [ ]:
# Cell 3: Initialize Session Logger
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = SessionLogger(session_id=session_id)

logger.print_success(f"Session started: {session_id}")
logger.print_info(f"Logs will be saved to: logs/{session_id}.log")

In [ ]:
# Cell 4: View Default Prompt
default_prompt = load_prompt_template('build-website')
print("Default Task Prompt:")
print("=" * 80)
print(default_prompt[:1000])  # Show first 1000 characters
print("...")
print(f"\nTotal length: {len(default_prompt)} characters")

In [ ]:
# Cell 5: Create CrewAI Crew
logger.print_section("Creating CrewAI Crew")

crew = create_coder_crew(verbose=True)

logger.print_success("Crew created successfully")
logger.print_info(f"Agents: {len(crew.agents)}")
logger.print_info(f"Tasks: {len(crew.tasks)}")